# DQN PyTorch
<img src="https://miro.medium.com/max/1206/1*nb61CxDTTAWR1EJnbCl1cA.png" 
     alt="Pseudo code" 
     style="width:800px;height:550px;"/>


In [3]:
import argparse
import torch
import torch.nn
import numpy as np
import random
import gym
from collections import namedtuple
from collections import deque
from typing import List, Tuple

Our environment is deterministic, so all equations presented here are also formulated deterministically for the sake of simplicity. In RL literature, they would also contain expectations over stochastic transitions in the environment. 

Loss function: Huber loss. Huber loss acts like the mean squared error when the error is small, but like the MAE when the error is large. This makes it more robust to ouliers when the estimates of Q are very noisy. We calculate this over a batch of transitions, B, sampled from the replay memory. 

In [24]:
class DQN(torch.nn.Module):
    def __init__(self, input_dim: int, output_dim: int, hidden_dim: int) -> None:
        """DQN Network
        Args:
            input_dim (int): `state` dimension.
                `state` is 2-D tensor of shape (n, input_dim)
            output_dim (int): Number of actions.
                Q_value is 2-D tensor of shape (n, output_dim)
            hidden_dim (int): Hidden dimension in fc layer
        """
        super(DQN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.PReLU()
        )

        self.layer2 = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.PReLU()
        )

        self.final = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Returns a Q_value
        Args:
            x (torch.Tensor): `State` 2-D tensor of shape (n, input_dim)
        Returns:
            torch.Tensor: Q_value, 2-D tensor of shape (n, output_dim)
        """
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.final(x)

        return x

In [25]:
Transition = namedtuple("Transition",
                        field_names=["state", "action", "reward", "next_state", "done"])


class ReplayMemory(object):

    def __init__(self, capacity: int) -> None:
        """Replay memory class
        Args:
            capacity (int): Max size of this memory
        """
        self.capacity = capacity
        self.cursor = 0
        self.memory = []

    def push(self,
             state: np.ndarray,
             action: int,
             reward: int,
             next_state: np.ndarray,
             done: bool) -> None:
        """Creates `Transition` and insert
        Args:
            state (np.ndarray): 1-D tensor of shape (input_dim,)
            action (int): action index (0 <= action < output_dim)
            reward (int): reward value
            next_state (np.ndarray): 1-D tensor of shape (input_dim,)
            done (bool): whether this state was last step
        """
        if len(self) < self.capacity:
            self.memory.append(None)

        self.memory[self.cursor] = Transition(state,
                                              action, reward, next_state, done)
        self.cursor = (self.cursor + 1) % self.capacity

    def pop(self, batch_size: int) -> List[Transition]:
        """Returns a minibatch of `Transition` randomly
        Args:
            batch_size (int): Size of mini-bach
        Returns:
            List[Transition]: Minibatch of `Transition`
        """
        return random.sample(self.memory, batch_size)

    def __len__(self) -> int:
        """Returns the length """
        return len(self.memory)

In [26]:
class Agent(object):

    def __init__(self, input_dim: int, output_dim: int, hidden_dim: int) -> None:
        """Agent class that choose action and train
        Args:
            input_dim (int): input dimension
            output_dim (int): output dimension
            hidden_dim (int): hidden dimension
        """
        self.dqn = DQN(input_dim, output_dim, hidden_dim)
        self.input_dim = input_dim
        self.output_dim = output_dim

        self.loss_fn = torch.nn.MSELoss()
        self.optim = torch.optim.Adam(self.dqn.parameters())

    def _to_variable(self, x: np.ndarray) -> torch.Tensor:
        """torch.Variable syntax helper
        Args:
            x (np.ndarray): 2-D tensor of shape (n, input_dim)
        Returns:
            torch.Tensor: torch variable
        """
        return torch.autograd.Variable(torch.Tensor(x))

    def get_action(self, states: np.ndarray, eps: float) -> int:
        """Returns an action
        Args:
            states (np.ndarray): 2-D tensor of shape (n, input_dim)
            eps (float): 𝜺-greedy for exploration
        Returns:
            int: action index
        """
        if np.random.rand() < eps:
            return np.random.choice(self.output_dim)
        else:
            self.dqn.train(mode=False)
            scores = self.get_Q(states)
            _, argmax = torch.max(scores.data, 1)
            return int(argmax.numpy())

    def get_Q(self, states: np.ndarray) -> torch.FloatTensor:
        """Returns `Q-value`
        Args:
            states (np.ndarray): 2-D Tensor of shape (n, input_dim)
        Returns:
            torch.FloatTensor: 2-D Tensor of shape (n, output_dim)
        """
        states = self._to_variable(states.reshape(-1, self.input_dim))
        self.dqn.train(mode=False)
        return self.dqn(states)

    def train(self, Q_pred: torch.FloatTensor, Q_true: torch.FloatTensor) -> float:
        """Computes `loss` and backpropagation
        Args:
            Q_pred (torch.FloatTensor): Predicted value by the network,
                2-D Tensor of shape(n, output_dim)
            Q_true (torch.FloatTensor): Target value obtained from the game,
                2-D Tensor of shape(n, output_dim)
        Returns:
            float: loss value
        """
        self.dqn.train(mode=True)
        self.optim.zero_grad()
        loss = self.loss_fn(Q_pred, Q_true)
        loss.backward()
        self.optim.step()

        return loss

In [27]:
def train_helper(agent: Agent, minibatch: List[Transition], gamma: float) -> float:
    """Prepare minibatch and train them
    Args:
        agent (Agent): Agent has `train(Q_pred, Q_true)` method
        minibatch (List[Transition]): Minibatch of `Transition`
        gamma (float): Discount rate of Q_target
    Returns:
        float: Loss value
    """
    states = np.vstack([x.state for x in minibatch])
    actions = np.array([x.action for x in minibatch])
    rewards = np.array([x.reward for x in minibatch])
    next_states = np.vstack([x.next_state for x in minibatch])
    done = np.array([x.done for x in minibatch])

    Q_predict = agent.get_Q(states)
    Q_target = Q_predict.clone().data.numpy()
    Q_target[np.arange(len(Q_target)), actions] = rewards + gamma * np.max(agent.get_Q(next_states).data.numpy(), axis=1) * ~done
    Q_target = agent._to_variable(Q_target)

    return agent.train(Q_predict, Q_target)

In [31]:
def play_episode(env: gym.Env,
                 agent: Agent,
                 replay_memory: ReplayMemory,
                 eps: float,
                 batch_size: int) -> int:
    """Play an epsiode and train
    Args:
        env (gym.Env): gym environment (CartPole-v0)
        agent (Agent): agent will train and get action
        replay_memory (ReplayMemory): trajectory is saved here
        eps (float): 𝜺-greedy for exploration
        batch_size (int): batch size
    Returns:
        int: reward earned in this episode
    """
    s = env.reset()
    done = False
    total_reward = 0

    while not done:

        a = agent.get_action(s, eps)
        s2, r, done, info = env.step(a)

        total_reward += r

        if done:
            r = -1
        replay_memory.push(s, a, r, s2, done)

        if len(replay_memory) > batch_size:

            minibatch = replay_memory.pop(batch_size)
            train_helper(agent, minibatch, GAMMA)

        s = s2

    return total_reward

In [29]:
def get_env_dim(env: gym.Env) -> Tuple[int, int]:
    """Returns input_dim & output_dim
    Args:
        env (gym.Env): gym Environment (CartPole-v0)
    Returns:
        int: input_dim
        int: output_dim
    """
    input_dim = env.observation_space.shape[0]
    output_dim = env.action_space.n

    return input_dim, output_dim

In [20]:
def epsilon_annealing( episode: int, max_episode: int, min_eps: float) -> float:
    ''' Returns ε-greedy
        1.0---|\
              | \
              |  \
        min_e +---+------->
                  |
                  max_episode
    Args:
        episode (int): Current episode (0<= episode)
        max_episode (int): After max episode, ε will be 'min_eps'
        min_eps (float): ε will never go below this value
    
    Returns:
        float: ε-value
    '''
    slope = (min_eps - 1.0)/ max_episode
    return max(slope * episode + 1.0, min_eps)
   
    

Best code ever written:
https://gym.openai.com/evaluations/eval_onwKGm96QkO9tJwdX7L0Gw/

In [33]:
# Main
GAME = 'CartPole-v0'
HIDDEN_DIM = 12
CAPACITY = 50000
NUM_EPISODES = 1000
MAX_EPISODES = 50
MIN_EPSILON = 0.01
BATCH_SIZE = 64
GAMMA = 0.99

try:
    env = gym.make(GAME)
    env = gym.wrappers.Monitor(env, directory='dqn_monitors', force=True)   # Write information about agent's performance in a file with optional video recording of agent in action 
    rewards = deque(maxlen=100)
    input_dim, output_dim = get_env_dim(env)
    agent = Agent(input_dim, output_dim, HIDDEN_DIM)
    replay_memory = ReplayMemory(CAPACITY)
    
    for i in range(NUM_EPISODES):
        eps = epsilon_annealing(i, MAX_EPISODES, MIN_EPSILON)
        r = play_episode(env, agent, replay_memory, eps, BATCH_SIZE)
        print("[Episode: {:5}] Reward: {:5} 𝜺-greedy: {:5.2f}".format(i + 1, r, eps))
        
        rewards.append(r)

        if len(rewards) == rewards.maxlen:
            if np.mean(rewards) >= 200:
                print("Game cleared in {} games with {}".format(i + 1, np.mean(rewards)))
                break
finally:
    env.close()

[Episode:     1] Reward:  11.0 𝜺-greedy:  1.00
[Episode:     2] Reward:  23.0 𝜺-greedy:  0.98
[Episode:     3] Reward:  13.0 𝜺-greedy:  0.96
[Episode:     4] Reward:  40.0 𝜺-greedy:  0.94
[Episode:     5] Reward:  16.0 𝜺-greedy:  0.92
[Episode:     6] Reward:  27.0 𝜺-greedy:  0.90
[Episode:     7] Reward:  16.0 𝜺-greedy:  0.88
[Episode:     8] Reward:  12.0 𝜺-greedy:  0.86
[Episode:     9] Reward:  16.0 𝜺-greedy:  0.84
[Episode:    10] Reward:  22.0 𝜺-greedy:  0.82
[Episode:    11] Reward:  15.0 𝜺-greedy:  0.80
[Episode:    12] Reward:  20.0 𝜺-greedy:  0.78
[Episode:    13] Reward:  14.0 𝜺-greedy:  0.76
[Episode:    14] Reward:  12.0 𝜺-greedy:  0.74
[Episode:    15] Reward:  25.0 𝜺-greedy:  0.72
[Episode:    16] Reward:  11.0 𝜺-greedy:  0.70
[Episode:    17] Reward:  13.0 𝜺-greedy:  0.68
[Episode:    18] Reward:  11.0 𝜺-greedy:  0.66
[Episode:    19] Reward:  15.0 𝜺-greedy:  0.64
[Episode:    20] Reward:  12.0 𝜺-greedy:  0.62
[Episode:    21] Reward:  17.0 𝜺-greedy:  0.60
[Episode:    

KeyboardInterrupt: 